In [27]:
import os
import pprint
from pymongo import MongoClient
import dotenv
from datetime import datetime as dt


1. Generamos la conexion a MongoDB

In [8]:
dotenv.load_dotenv(dotenv.find_dotenv())
PORT = os.environ.get('MONGO_PORT')

string_mongo = f"mongodb://localhost:27017"

mongodm = MongoClient(string_mongo)

print(mongodm.list_database_names())


['admin', 'config', 'local', 'miDB']


1.2 Creamos una nueva BD __production__ y le creamos dos colecciones __books__ y __authors__

In [20]:
mongodm.get_database('production')
production = mongodm.get_database('production')

try:
    production.create_collection("Book")
    production.create_collection("Author")
except Exception as e:
    print(e)

collection Book already exists


1.3 Definimos el esquema para __book__ y modificamos la coleccion.

In [17]:
book_validator = {
    "$jsonSchema":{
        "bsonType":"object",
        "required":["title", "authors", "publish_date", "type", "copies"],
        "properties":{
            "title":{
                "bsonType": "string",
                "description": "must be a string and is required"
            },
            "authors":{
                "bsonType": "array",
                "items": {
                    "bsonType": "objectId",
                    "description": "must be an objectId and is required"
                }
            },
            "published_date": {
                "bsonType": "date",
                "description": "must be a date and is required"
            },
            "type": {
                "enum": ["Fiction", "Non-Fiction"],
                "description": "can only be one of the enum values and its required"
            },
            "copies": {
                "bsonType": "int",
                "minimum": 0,
                "description": "should be greather than 0 and is required"
            }

        }
    }
}

In [18]:
production.command("collMod", "Book", validator=book_validator)

{'ok': 1.0}

1.4 Definimos el esquema para __author__ y modificamos la coleccion.

In [30]:
author_validator={
    "$jsonSchema":{
        "bsonType":"object",
        "required":["first_name", "last_name", "date_of_birth"],
        "properties":{
            "first_name":{
                "bsonType": "string",
                "description": "must be a string and is required"
            },
            "last_name":{
                "bsonType": "string",
                "description": "must be a string and is required"
            },
            "date_of_birth": {
                "bsonType": "date",
                "description": "must be a date and is required"
            },
        }
    }
}

In [31]:
production.command("collMod", "Author", validator=author_validator)

{'ok': 1.0}

2.1 Insertamos datos en ambas colecciones usando __bulk_insert__

In [34]:
def create_data():
    authors = [
        {
            "first_name": "Nicolas",
            "last_name": "leali",
            "date_of_birth": dt(1988, 4,1)
        },
        {
            "first_name": "George",
            "last_name": "Orwell",
            "date_of_birth": dt(1903, 6,25)
        },
        {
            "first_name": "Herman",
            "last_name": "Melville",
            "date_of_birth": dt(1818, 8,1)
        },
        {
            "first_name": "F. Scott",
            "last_name": "Fitzgerald",
            "date_of_birth": dt(1896, 9,24)
        }
    ]
    
    authors_collection = production.get_collection('Author')
    authors = authors_collection.insert_many(authors).inserted_ids
    
    books = [
        {
            "title": "MondoDB Advanced Tutorial",
            "authors": [authors[0]],
            "publish_date": dt.today(),
            "type": "Non-Fiction",
            "copies": 5
        },
        {
            "title": "Python for Dummies",
            "authors": [authors[0]],
            "publish_date": dt(2022,1,27),
            "type": "Non-Fiction",
            "copies": 5
        },
        {
            "title": "Nineteen Eigthy-Four",
            "authors": [authors[1]],
            "publish_date": dt(1949,6,8),
            "type": "Fiction",
            "copies": 5
        },
        {
            "title": "The Great Gatsby",
            "authors": [authors[3]],
            "publish_date": dt(2013,5,23),
            "type": "Fiction",
            "copies": 5
        },
          {
            "title": "Moby Dick",
            "authors": [authors[2]],
            "publish_date": dt(1851,9,24),
            "type": "Fiction",
            "copies": 5
        }
    ]
    
    book_collection = production.get_collection('Book')
    book_collection.insert_many(books)

In [35]:
#create_data()

3.1 Usamos __regex__ para buscar los titulos que contienen al menos una __A__

In [41]:
def contiene_una_a():
    printer = pprint.PrettyPrinter()
    
    books_collection = production.get_collection('Book')
    titulos = books_collection.find({"title":{"$regex": "a{1}"}})
    printer.pprint(list(titulos))
    
contiene_una_a()

[{'_id': ObjectId('65dd2af305ee172005884912'),
  'authors': [ObjectId('65dd2af305ee17200588490e')],
  'copies': 5,
  'publish_date': datetime.datetime(2024, 2, 26, 21, 21, 7, 176000),
  'title': 'MondoDB Advanced Tutorial',
  'type': 'Non-Fiction'},
 {'_id': ObjectId('65dd2af305ee172005884915'),
  'authors': [ObjectId('65dd2af305ee172005884911')],
  'copies': 5,
  'publish_date': datetime.datetime(2013, 5, 23, 0, 0),
  'title': 'The Great Gatsby',
  'type': 'Fiction'}]


3.2 Obtenemos todos los libros escritos por los autores.

In [46]:
printer = pprint.PrettyPrinter()
authors_collection = production.get_collection('Author')

authors_and_books = authors_collection.aggregate([{
    "$lookup":{
        "from": "Book",
        "localField": "_id",
        "foreignField": "authors",
        "as": "books"
    }
}])

printer.pprint(list(authors_and_books))

[{'_id': ObjectId('65dd2af305ee17200588490e'),
  'books': [{'_id': ObjectId('65dd2af305ee172005884912'),
             'authors': [ObjectId('65dd2af305ee17200588490e')],
             'copies': 5,
             'publish_date': datetime.datetime(2024, 2, 26, 21, 21, 7, 176000),
             'title': 'MondoDB Advanced Tutorial',
             'type': 'Non-Fiction'},
            {'_id': ObjectId('65dd2af305ee172005884913'),
             'authors': [ObjectId('65dd2af305ee17200588490e')],
             'copies': 5,
             'publish_date': datetime.datetime(2022, 1, 27, 0, 0),
             'title': 'Python for Dummies',
             'type': 'Non-Fiction'}],
  'date_of_birth': datetime.datetime(1988, 4, 1, 0, 0),
  'first_name': 'Nicolas',
  'last_name': 'leali'},
 {'_id': ObjectId('65dd2af305ee17200588490f'),
  'books': [{'_id': ObjectId('65dd2af305ee172005884914'),
             'authors': [ObjectId('65dd2af305ee17200588490f')],
             'copies': 5,
             'publish_date': datetime

3.3 Obtenemos la cantidad de libros por autor

In [62]:
printer = pprint.PrettyPrinter()
count_books_by_author = authors_collection.aggregate(
    [ # lista
    { # primer pipeline left-join
        "$lookup":{
            "from": "Book",
            "localField": "_id",
            "foreignField": "authors",
            "as": "books"
        }
    },
    { # segundo pipeline - count
        "$addFields":{
            "total_libros":{"$size":"$books"}
        }
    },
    { # tercer pipeline - muestra
        "$project":{
            "first_name":1, "last_name":1, "total_libros":1, "_id":0
        }
    }
])

printer.pprint(list(count_books_by_author))

[{'first_name': 'Nicolas', 'last_name': 'leali', 'total_libros': 2},
 {'first_name': 'George', 'last_name': 'Orwell', 'total_libros': 1},
 {'first_name': 'Herman', 'last_name': 'Melville', 'total_libros': 1},
 {'first_name': 'F. Scott', 'last_name': 'Fitzgerald', 'total_libros': 1}]


3.4 Mostramos los libros cuyos autores estén entre cierto rango de edad.

In [75]:
printer = pprint.PrettyPrinter()
book_collection = production.get_collection('Book')
books_and_authors_by_age = book_collection.aggregate(
    [ # Lista del pipeline
     { # pipeline 1 - lookup
      
        "$lookup":{
            "from": "Author",
            "localField": "authors",
            "foreignField": "_id",
            "as": "autores"
        }
         
     },
     { # pipeline 2 - set edad
      
      "$set":{
          "autores":{
              "$map":{
                  "input": "$autores",
                  "in": {
                      "age":{
                          "$dateDiff":{
                              "startDate": "$$this.date_of_birth",
                              "endDate": "$$NOW",
                              "unit": "year"
                          }
                      },
                      "first_name":"$$this.first_name",
                      "last_name": "$$this.last_name"
                  }
              }
          }
      }
          
         
     },
     
     { # piplene 3 - filtrar por cierta edad
      
      "$match": {
          "$and": [
              {"autores.age": {"$gte":18}},
              {"autores.age": {"$lte":100}}
          ]
      }
         
     },
     {# pipeline 4 - sort by age
      
      "$sort":{
          "age":1
            }
         
     }
]
    )

printer.pprint(list(books_and_authors_by_age))

[{'_id': ObjectId('65dd2af305ee172005884912'),
  'authors': [ObjectId('65dd2af305ee17200588490e')],
  'autores': [{'age': 36, 'first_name': 'Nicolas', 'last_name': 'leali'}],
  'copies': 5,
  'publish_date': datetime.datetime(2024, 2, 26, 21, 21, 7, 176000),
  'title': 'MondoDB Advanced Tutorial',
  'type': 'Non-Fiction'},
 {'_id': ObjectId('65dd2af305ee172005884913'),
  'authors': [ObjectId('65dd2af305ee17200588490e')],
  'autores': [{'age': 36, 'first_name': 'Nicolas', 'last_name': 'leali'}],
  'copies': 5,
  'publish_date': datetime.datetime(2022, 1, 27, 0, 0),
  'title': 'Python for Dummies',
  'type': 'Non-Fiction'}]
